In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## Load the data
We will be doing some joins on the data in this notebook, so we drop some columns.

In [ ]:
stops = gpd.read_file("data/metlink-stops.gpkg").to_crs(2193)[
    ["stop_code", "stop_name", "geometry"]]

routes = gpd.read_file("data/metlink-routes.gpkg").to_crs(2193)[
    ["route_id", "route_type", "route_short_name", "geometry"]]

sa2 = gpd.read_file("data/sa2-wellington.gpkg")

# Binary GIS operations
Whenever possible, I recommend performing joins, filtering data, querying data, etc. without using spatial characteristics. It's just quicker. This holds whether you are using a GIS or using Python. The speed of tabular operations based on non-spatial data is one reason discrete global grid systems such as H3 are gaining in popularity.

Anyway... regardless there are many situations where you do want to deploy spatial methods which we quickly explore in this notebook.

## The spatial predicate methods
'Spatial predicates' is a fancy word for 'tests for spatial relations'. These are things like 'intersects', 'contains' and so on. A full list of the predicates available as `GeoSeries` methods is in the documentation [here](https://geopandas.org/en/stable/docs/reference/geoseries.html#binary-predicates).

But, I'll be honest... the binary spatial predicate operations in `geopandas` are _weird_. They work fine, but they work _row-wise_. This means that when you do something like (say)

```python
sa2.geometry.intersects(routes.geometry)
```

which, usefully, can be shortcutted as

```python
sa2.intersects(routes)
```

what is being done is a row-by-row test if the first polygon in the `sa2` table intersects the first route in the `routes` table, and so on. As illustrated in the `geopandas` documentation, it does this:

![](binary_op-02.svg)

There is an `align` option you can set to make the one-to-one comparison based on the respective dataframe indexes, but either way it's hard to think of a situation where this one-to-one behaviour is useful! Generally, what you want to do is check for spatial relationships between every possible pair of geometries in the two dataframes, so that you can identify which pairs of elements from the two datasets are related to one another. This is something that the binary predicate methods of `geopandas` `GeoSeries`  explicitly don't support. As noted in the documentation pages "This method works in a row-wise manner. It does not check if an element of one GeoSeries `<insert-method-here>` _any_ element of the other one."

To illustrate this, take the simple example of applying a _spatial filter_. It is a common requirement to filter a dataset (say the bus routes) to only those that pass overlap or intersect, or otherwise spatially relate to at least one of the SA2s in our Wellington dataset.  Using the `intersects()` method as a boolean selector without thinking too hard about it we get

In [ ]:
m = sa2.explore(
    style_kwds = {"fillColor": "grey", "color": "white", "weight": 1}, 
    tooltip = False, tiles = "CartoDB Positron")

routes.explore(
    m = m, style_kwds = {"color": "black"}, tooltip = False)

routes[routes.intersects(sa2)].explore(
    m = m, style_kwds = {"color": "red"}, tooltip = False)

Clearly, this misses many routes that intersect with SA2 polygons, and that's almost certainly _not_ what you want. To get the filtering behaviour I had in mind, I must apply `union_all()` to the SA2 data, like this:

In [ ]:
m = sa2.explore(
    style_kwds = {"fillColor": "grey", "color": "white", "weight": 1}, 
    tooltip = False, tiles = "CartoDB Positron")

routes.explore(
    m = m, style_kwds = {"color": "black"}, tooltip = False)

routes[routes.intersects(sa2.union_all())].explore(
    m = m, style_kwds = {"color": "red"}, tooltip = False)

OK... fine, it works. But it's definitely strange! If you are going to do a lot of location based queries of this kind you probably should make the `union_all()` result as a new `GeoSeries` for repeated use.

An additional point to note here is that combining filters requires that you make sure the order of application of the boolean logic is correct. The easiest way to manage this is with lots of parentheses. Remember to use `&` and `|` here for the logical and/or operations. Below I select only bus routes (`route_type == "3"`) that intersect the SA2s.

In [ ]:
sa2_u = sa2.union_all()
bus_routes_sa2 = routes[(routes.intersects(sa2_u)) &
                        (routes.route_type == "3")]

bus_routes_sa2.explore(tiles = "CartoDB Positron")

The `union_all()` operation can be computationally costly, and it may be quicker in some cases to apply it not to the raw data but to the _envelopes_ of the data, making the unioning operation that combines all the polygons quicker.

In [ ]:
routes[(routes.geometry.intersects(sa2.envelope.union_all())) &
       (routes.route_type == "3")].explore(tiles = "CartoDB Positron")

Now that we have a reduced set of routes, we can do a similar filter on the stops so that only those within some distance of one of the selected routes are retained. For this we change the `predicate` of the boolean selection.

In [ ]:
bus_stops_sa2 = \
    stops[stops.dwithin(bus_routes_sa2.union_all(), distance = 10)]

m = bus_routes_sa2.explore(style_kwds = {"color": "black"}, 
                           tiles = "CartoDB Positron", tooltip = False)
bus_stops_sa2.explore(m = m, style_kwds = {"color": "red"}, tooltip = False)


## Clipping
An alternative to spatial filtering may be _clipping_ a layer to the extent of another layer. This is straightforward, and will automatically clip to the total bounds of the clipping layer. This is generally most useful for mapping purposes, when you want to restrict all the layers in a map to match the extent of one of the layers.

In [ ]:
ax = gpd.GeoSeries([sa2_u]).plot(figsize = (8, 8), fc = "lightgrey")
routes.clip(sa2).plot(ax = ax, ec = "k", lw = 0.5).set_axis_off()

## Spatial join
As we saw in `pandas` a fundamental analysis operation is reorganising data by joining tables based on matching values in data columns. The same operation is available when the relation between the columns is a spatial one, provided both datasets have associated geometry, and provided that they are in matching projections. Because a spatial join is always based on a geometric relationship you don't have to supply any information about the columns to use (it will always use the geometry columns). 

The join is assumed to be `how = "inner"` so only combinations of geometries for which the relationship exists will be returned. Often you will want to retain all the records in the 'left' data table and should specify `how = "left"` instead. 

You can also specify different spatial relations of interest using the `predicate` parameter. This defaults to `"intersects"` but you can select any of the predicates returned by `df.sindex.valid_query_predicates`.

So... let's first check which predicates are available.

In [ ]:
sa2.sindex.valid_query_predicates

Seems reasonable. From what I can tell it returns the same list for line and point layers, which seems a bit wrong, but let's carry on anyway.

Some relationships are **one-to-one** and relatively simple. For example, each bus stop can only be within one SA2.

In [ ]:
bus_stops_sa2.sjoin(sa2, how = "inner", predicate = "within")

The result of the join retains the geometry of the left hand dataset and attaches the data from the row in the right hand dataset with which it has the specified spatial relationship.

**One-to-many** relationships are more complicated. For example, each SA2 contains anything from 0 to many bus stops.

In [ ]:
sa2_stops = sa2.sjoin(bus_stops_sa2, how = "left")
sa2_stops

We have an output table with 1272 entries. This is more than the numbers of SA2s (78) and also more than the number of stops (1268). The four 'extra' rows are four SA2s which contain no stops, which are retained because we specified `how = "left"`. They appear in the table with NaN values in the columns from the stops dataset. An example is Tawa North as seen above.

With a one-to-many relation like this we likely need to apply some additional processing so that the added columns contain meaningful data for each SA2 probably based on summary statistics on groups based on an id variable for the left hand data table. We can do this using any function or set of functions as specified after a `groupby().agg()` combination.

More complicated still is a **many-to-many** relationship which may also incidentally lead to rapid increases in the size of the data as `geopandas` creates duplicate copies of data for every possible pair of geometries from the two input datasets that are related in the requested way. For example, a spatial join based of SA2 data to routes based on the intersect predicates gives us 

In [ ]:
bus_routes_sa2.sjoin(sa2, how = "left", predicate = "intersects")

This has seen the 318 routes expand out to almost 4000 intersection relations between bus routes and SA2s. Again, a `groupby()` operation is needed for the overall result to make much sense. For example to sum the populations of SA2s along each bus route we could do something like the following.

In [ ]:
bus_route_pops = bus_routes_sa2 \
    .sjoin(sa2, how = "left", predicate = "intersects") \
    .groupby("route_short_name", as_index = False) \
    .aggregate({"geometry": "first", "CURPop": "sum"})

bus_route_pops.head()

It is important to note that although this result looks like a `GeoDataFrame`, it may need some repair work done for it to behave as one

In [ ]:
bus_route_pops.plot().set_axis_off()

In [ ]:
bus_route_pops = bus_route_pops.set_geometry("geometry")

bus_route_pops.plot().set_axis_off()

All this means that a better overall approach to that set out above may be as shown below where we make the joined dataset as minimal as possible (just the `route_short_name` and `CURPop` columns) and then join it back to the original `GeoDataFrame` with a `DataFrame.merge()`. This way the `GeoDataFrame` never stops being a `GeoDataFrame`.

In [ ]:
bus_route_pops = bus_routes_sa2 \
    .sjoin(sa2, how = "left", predicate = "intersects") \
    .groupby("route_short_name", as_index = False) \
    .aggregate({"CURPop": "sum"})

bus_route_pops = bus_routes_sa2 \
    .merge(bus_route_pops)

bus_route_pops.plot().set_axis_off()

### `as_index`
Above when using `groupby()` I have specified `as_index = False`. This is so that `pandas` retains the grouping variable as an attribute in the data and doesn't make it into the index of the resulting `(Geo)DataFrame`. This is more likely to be the preferred option in many geospatial workflows where we continue to need access to all attributes as potential join variables.

In [ ]:
bus_routes_sa2 \
    .sjoin(sa2, how = "left", predicate = "intersects") \
    .groupby("route_short_name") \
    .aggregate({"geometry": "first", "CURPop": "sum"}) \
    .head()

### Aside: spatial joins as a spatial filter
An odd side-effect of how the `geopandas` API is organised is that there are some cases where a spatial join may be the quickest way to apply a spatial filter. Compare the timings of the two approaches below.

In [ ]:
stops[stops.intersects(sa2.union_all())]

In [ ]:
stops.sjoin(sa2[["geometry"]])

I haven't conducted extensive testing of this idea, but it works well for one-to-one relationships. It is more complicated when the spatial relations are less straightforward, and some `groupby()` tidying up is needed (see below), but `sjoin` still appears faster than applying the `GeoSeries` spatial predicate methods in general.

In [ ]:
routes[routes.intersects(sa2_u)] \
    .groupby("route_short_name", as_index = False) \
    .agg("first")

In [ ]:
routes.sjoin(sa2[["geometry"]]) \
    .groupby("route_short_name", as_index = False) \
    .agg("first")